# Gradio (ML Models interfaces)

In [1]:
!pip install gradio
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.9/287.9 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.2/137.2 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 

## Image Classification

In [2]:
import gradio as gr
import torch
from torchvision.models import resnet50, ResNet50_Weights
from torchvision import transforms
from PIL import Image

weights = ResNet50_Weights.DEFAULT
img_model = resnet50(weights=weights)

def inference(data):
    img = Image.fromarray(data.astype('uint8'), 'RGB')
    img_model.eval()
    preprocess = weights.transforms()
    batch = preprocess(img).unsqueeze(0)
    prediction = img_model(batch).squeeze(0).softmax(0)
    val, ind = torch.topk(prediction, 5)
    categories = []
    for i in ind:
      categories.append(weights.meta["categories"][i])
    return dict(zip(categories, map(float, val)))

gr.Interface(fn=inference, 
             inputs=gr.Image(shape=(224, 224)), 
             outputs=gr.Label(num_top_classes=5)).launch(share=True) #, debug=True Use in Colab

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 83.1MB/s]


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://0c8e6d9159a27e5978.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


## Text Generation

In [3]:
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

# Loading and setting up GPT2 Open AI Transformer Model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
txt_model = TFGPT2LMHeadModel.from_pretrained("gpt2", 
                                          pad_token_id=tokenizer.eos_token_id)

def generate_text(inp):
    # Encoding the starting point of the sentence we want to predict
    input_data = tokenizer.encode(inp, 
                                  return_tensors='tf')
    # Generating Output String
    output = txt_model.generate(
        input_data, 
        do_sample=True, 
        max_length=50, 
        top_k=30
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

gr.Interface(generate_text,
             "textbox", 
             gr.Textbox()).launch(share=True) #, debug=True Use in Colab

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://d9e6412e576d57a75e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


# Exercise

So far we have explored how to use Gradio for an Image and Text, problem. How about working with tabular data? Can you come up with a simple example on how Gradio could be useful in these kind of situations? Feel free to explore [Gradio Official Documentation](https://gradio.app/) for any help.